OpenAI Agents SDK FrameWork - Sales Agents

Details: <a href="https://openai.github.io/openai-agents-python/">https://openai.github.io/openai-agents-python/</a>

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [ ]:
load_dotenv(override=True)

In [ ]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
sendgrid_api_key = os.getenv('SENDGRID_API_KEY')
sendgrid_mailfrom = os.getenv('SENDGRID_MAILFROM')
sendgrid_mailto = os.getenv('SENDGRID_MAILTO')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if sendgrid_api_key:
    print(f"SendGrid API Key exists and begins {sendgrid_api_key[:4]}")
else:
    print("SendGrid API Key not set")

if sendgrid_mailfrom:
    print(f"MailFrom Key exists and begins {sendgrid_mailfrom[:4]}")
else:
    print("MailFrom  Key not set")

if sendgrid_mailto:
    print(f"MailTo Key exists and begins {sendgrid_mailto[:4]}")
else:
    print("MailTo Key not set")

In [ ]:
# Let's just check emails are working for you. 202 responce means delivery OK

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(os.environ.get('SENDGRID_MAILFROM'))  # Change to your verified sender
    to_email = To(os.environ.get('SENDGRID_MAILTO'))  # Change to your recipient
    content = Content("text/plain", "Your test email delivered")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

In [ ]:
# Make an agent with name, instructions, model

agent = Agent(name="JokeTeller", instructions="You are a joke teller", model="gpt-4o-mini")

In [ ]:
agent

In [ ]:
# Run the joke with Runner.run(agent, prompt) then print final_output

with trace("JokerTeller Outputs"):
    result = await Runner.run(agent, "Tell a random joke about artificial Intelligence")
    print(result.final_output)